In [3]:
pip install google-cloud-language pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 14.2 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "civic-network-418910-0aa3df60f95d.json"

In [8]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import language_v1

def fetch_comments_with_sentiment_google_api(stock_code, start_page, end_page):
    client = language_v1.LanguageServiceClient()
    all_comments = []

    for page_num in range(start_page, end_page + 1):
        url = f"http://guba.eastmoney.com/list,{stock_code},f_{page_num}.html"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            comments = soup.find_all('tr', class_='listitem')
            for comment in comments:
                read = comment.find('div', class_='read').text if comment.find('div', class_='read') else 'N/A'
                reply = comment.find('div', class_='reply').text if comment.find('div', class_='reply') else 'N/A'
                title_div = comment.find('div', class_='title')
                title = title_div.get_text(strip=True) if title_div else 'N/A'
                author = comment.find('div', class_='author').text if comment.find('div', class_='author') else 'N/A'
                update_time = comment.find('div', class_='update').text if comment.find('div', class_='update') else 'N/A'

                sentiment_score = get_sentiment_score_google_api(client, title)

                all_comments.append({
                    "页码": page_num,
                    "阅读": read,
                    "回复": reply,
                    "标题": title,
                    "作者": author,
                    "更新时间": update_time,
                    "情绪得分": sentiment_score
                })
        else:
            print(f"请求失败，状态码：{response.status_code}")

    df = pd.DataFrame(all_comments)
    csv_file = f"comments_{stock_code}_p{start_page}_to_p{end_page}_score.csv"
    df.to_csv(csv_file, index=False, encoding='utf_8_sig')
    print(f"数据已保存到CSV文件：{csv_file}")

def get_sentiment_score_google_api(client, text):
    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    # Google Cloud的情绪分数范围是[-1, 1]，我们将它调整到[0, 10]的范围
    sentiment_score = (sentiment.score + 1) * 5
    return round(sentiment_score, 1)

# 示例：爬取宁德时代股吧的第1页到第2页的评论，并保存到CSV，包含情绪得分
fetch_comments_with_sentiment_google_api('300750', 1, 2)


数据已保存到CSV文件：comments_300750_p1_to_p2_score.csv
